In [32]:
from networkx_graph import RepresentativesGraph
import networkx as nx # Para redes en general
from networkx.readwrite import json_graph
from new_disparity_filter import NewDisparityFilter
from procesador_votaciones import ProcesadorDeVotaciones
from heavy_graph_maker import HeavyGraphMaker

import matplotlib.pylab as plt # Para graficar networkx y demases
import matplotlib.cm as cm # para mapas de colores
import numpy as np
from sklearn.metrics import normalized_mutual_info_score as mis # Función para calcular información mutua
import cairo # Importo Cairo
import funciones as f
import numpy as np

In [8]:
years = list(range(1993,2021))
procesador = ProcesadorDeVotaciones(years[0],years[-1], 'data')
congress = procesador.procesar()

#red_igraph = ig.Graph.TupleList(red.edges(), directed=False)


Loading from data/1993/1
Getting data/1993/2 from API
Getting data/1993/3 from API
Getting data/1993/4 from API
Getting data/1993/5 from API
Getting data/1993/6 from API
Getting data/1993/7 from API
Getting data/1993/8 from API
Getting data/1993/9 from API
Getting data/1993/10 from API
Getting data/1993/11 from API
Getting data/1993/12 from API
Loading from data/1994/1
Loading from data/1994/2
Loading from data/1994/3
Loading from data/1994/4
Loading from data/1994/5
Loading from data/1994/6
Loading from data/1994/7
Loading from data/1994/8
Loading from data/1994/9
Loading from data/1994/10
Loading from data/1994/11
Loading from data/1994/12
Loading from data/1995/1
Loading from data/1995/2
Loading from data/1995/3
Loading from data/1995/4
Loading from data/1995/5
Loading from data/1995/6
Loading from data/1995/7
Loading from data/1995/8
Loading from data/1995/9
Loading from data/1995/10
Loading from data/1995/11
Loading from data/1995/12
Loading from data/1996/1
Loading from data/1996

Loading from data/2019/12
Loading from data/2020/1
Loading from data/2020/2
Loading from data/2020/3
Loading from data/2020/4
Loading from data/2020/5
Loading from data/2020/6
Loading from data/2020/7
Loading from data/2020/8
Loading from data/2020/9
Loading from data/2020/10
Loading from data/2020/11
Loading from data/2020/12


In [28]:
alphas  = {}
alphas[1993] = 0.185
alphas[1994] = 0.185
alphas[1995] = 0.21 #dudoso
alphas[1996] = 0.2
alphas[1997] = 0.19
alphas[1998] = 0.24
alphas[1999] = 0.25 #dudoso, GC = 0.8
alphas[2000] = 0.25
alphas[2001] = 0.22
alphas[2002] = 0.17
alphas[2003] = 0.17
alphas[2004] = 0.26 
alphas[2005] = 0.21
alphas[2006] = 0.145
alphas[2007] = 0.13
alphas[2008] = 0.222
alphas[2009] = 0.2
alphas[2010] = 0.22
alphas[2011] = 0.19 #rari
alphas[2012] = 0.231 #many edges
alphas[2013] = 0.32 #5k de edges
alphas[2014] = 0.22
alphas[2015]  = 0.2 
alphas[2016] = 0.26
alphas[2017] = 0.19 #hay un grupo muy separado que si nos vamos a 0.185 se sale de la GC
alphas[2018] = 0.25
alphas[2019] = 0.17
alphas[2020] = 0.3 #9k muy loco, hermoso

In [34]:
def get_GC(year,alpha,congreso):
    maker = HeavyGraphMaker(congreso)
    grafo = maker.create_year_network(year, positive = True)
    copia = grafo.copy()
    grafo_cortado = NewDisparityFilter(copia).alpha_cut(alpha_t = alpha)
    GC_cut, _ = grafo_cortado.gigant_component()
    return GC_cut

In [21]:
def get_ideologies_dict(graph,year):
    dict_ideologies = {}
    for ideology in file[str(year)]:
        dict_ideologies[ideology] = {}
        nodes = []
        for node in graph.get_nodes():
            if list(node.parties[year])[0].text in file[str(year)][ideology]:
                nodes.append(node)
        dict_ideologies[ideology] = nodes
    return dict_ideologies

In [22]:
def plot_graph_ideologies(graph,year):
    nodes = graph.get_nodes()
    grupos = list(get_ideologies_dict(graph,year).keys())
    mapping = dict(zip((set(grupos)),count()))
    colors = [mapping[grupos[i]] for i,n in enumerate(nodes)]
    
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph, pos, nodelist=nodes, node_color=colors,  with_labels=False, node_size=10, cmap=plt.cm.jet)
    nx.draw_networkx_edges(graph, pos, alpha=0.05)

In [23]:
def get_weights(graph):
    new_weights = []
    for e in graph.get_edges():
        new_weights.append(graph.graph[e[0]][e[1]]['weight'])
    return new_weights

In [45]:
import funciones as f
from collections import OrderedDict

def get_community_dict(particiones):
    dict_community = {}
    for part in particiones:
        dict_community[part] = {}
        keys = list(OrderedDict.fromkeys(list(particiones[part].values())))
        values = []
        for key in keys:
            values.append([i for i,j in particiones[part].items() if j == key])

        dict_community[part]= {k: values[k] for k in keys}
    return dict_community

In [60]:
def intersection(dict_comm,dict_ideo, dict_a_guardar):
    for algo_i in dict_comm: #recorro las particiones delos distintos algos
        dict_a_guardar[algo_i] = {}
        for particion in dict_comm[algo_i]:
            dict_a_guardar[algo_i]['Partición '+str(particion)] = {}
            a = set(dict_comm[algo_i][particion])
            
            #le agrego el tamanio de cada comunidad como variable
            dict_a_guardar[algo_i]['Partición '+str(particion)]['Tamaño'] = {}
            dict_a_guardar[algo_i]['Partición '+str(particion)]['Tamaño'] = len(dict_comm[algo_i][particion])
            
            for partido in dict_ideo:
                interseccion = a.intersection(set(dict_ideo[partido]))
                dict_a_guardar[algo_i]['Partición '+str(particion)][partido] = {}
                dict_a_guardar[algo_i]['Partición '+str(particion)][partido] = len(interseccion) / len(a)
    return dict_a_guardar

In [26]:
import yaml
file = open('configs/ideologies.json').read()
file = yaml.load(file)

C:\Users\Luna\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
year = 2019
alpha = 0.17

In [37]:
grafo = get_GC(year,alpha,congress)
dict_ideo = get_ideologies_dict(grafo,year)
dict_ideo

The graph has 25774 edges
Max alpha is 0.9735252711482005, min alpha is 0.029449692694460218
Enlaces sacados: 1000
Enlaces sacados: 2000
Enlaces sacados: 3000
Enlaces sacados: 4000
Enlaces sacados: 5000
Enlaces sacados: 6000
Enlaces sacados: 7000
Enlaces sacados: 8000
Enlaces sacados: 9000
Enlaces sacados: 10000
Enlaces sacados: 11000
Enlaces sacados: 12000
Enlaces sacados: 13000
Enlaces sacados: 14000
Enlaces sacados: 15000
Enlaces sacados: 16000
Enlaces sacados: 17000
Enlaces sacados: 18000
Enlaces sacados: 19000
Enlaces sacados: 20000
20382 edges deleted, 5392 left. 20.92% left.
Gigant component is 0.9420849420849421 of the total


{'Justicialismo': [<representative.Representative at 0x252610a5278>,
 'Cambiemos': [<representative.Representative at 0x252610016d8>,
 'Peronismo disidente': [<representative.Representative at 0x2525fd6e4a8>,
  <representative.Representative at 0x252610b6f28>]}

In [ ]:
plot_graph_ideologies(grafo,year)

In [47]:
array_funciones = [f.fast_greedy,f.infomap,f.leiden1]
particiones = {}
for i,metodo in enumerate(array_funciones):
    particiones[metodo] = {}
    particion = metodo(grafo.graph,get_weights(grafo))
    particiones[metodo] = particion

In [61]:
dict_comm = get_community_dict(particiones)
dict_a_guardar = {}
intersection(dict_comm,dict_ideo, dict_a_guardar)

{<function funciones.fast_greedy(Red_recableada, w)>: {'Partición 0': {'Tamaño': 120,
   'Justicialismo': 0.525,
   'Cambiemos': 0.016666666666666666,
   'Peronismo disidente': 0.15},
  'Partición 1': {'Tamaño': 22,
   'Justicialismo': 0.0,
   'Cambiemos': 0.0,
   'Peronismo disidente': 0.4090909090909091},
  'Partición 2': {'Tamaño': 102,
   'Justicialismo': 0.0,
   'Cambiemos': 0.9509803921568627,
   'Peronismo disidente': 0.0}},
 <function funciones.infomap(Red_recableada, w)>: {'Partición 0': {'Tamaño': 98,
   'Justicialismo': 0.6326530612244898,
   'Cambiemos': 0.0,
   'Peronismo disidente': 0.08163265306122448},
  'Partición 1': {'Tamaño': 102,
   'Justicialismo': 0.0,
   'Cambiemos': 0.9509803921568627,
   'Peronismo disidente': 0.0},
  'Partición 2': {'Tamaño': 44,
   'Justicialismo': 0.022727272727272728,
   'Cambiemos': 0.045454545454545456,
   'Peronismo disidente': 0.4318181818181818}},
 <function funciones.leiden1(red, w)>: {'Partición 0': {'Tamaño': 102,
   'Justicialismo

In [62]:
import pandas as pd
#fastgreedy algo
nodos_df = dict_a_guardar[list(dict_a_guardar.keys())[0]].copy() #no modificar data
df = pd.DataFrame(nodos_df).transpose()
df = df.astype(float).round(3)

df

,Cambiemos,Justicialismo,Peronismo disidente,Tamaño
Partición 0,0.017,0.525,0.150,120.0
Partición 1,0.000,0.000,0.409,22.0
Partición 2,0.951,0.000,0.000,102.0
